#保存和恢复模型

模型进度可在训练期间和之后保存。这意味着，您可以从上次暂停的地方继续训练模型，避免训练时间过长。此外，可以保存意味着您可以分享模型，而他人可以对您的工作成果进行再创作。发布研究模型和相关技术时，大部分机器学习从业者会分享以下内容：

- 用于创建模型的代码，以及
- 模型的训练权重或参数

分享此类数据有助于他人了解模型的工作原理并尝试使用新数据自行尝试模型。

>注意：请谨慎使用不可信的代码 - TensorFlow 模型就是代码。有关详情，请参阅安全地使用 TensorFlow。

##选项
您可以通过多种不同的方法保存 TensorFlow 模型，具体取决于您使用的 API。本指南使用的是 tf.keras，它是一种用于在 TensorFlow 中构建和训练模型的高阶 API。要了解其他方法，请参阅 TensorFlow 保存和恢复指南或在 Eager 中保存。

##设置
###安装和导入
安装并导入 TensorFlow 和依赖项：

In [0]:
!pip install -q h5py pyyaml

##获取示例数据集
我们将使用 MNIST 数据集训练模型，以演示如何保存权重。要加快演示运行速度，请仅使用前 1000 个样本：

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'1.14.0'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/data/models')
!pwd

/content/drive/My Drive/data/models


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) /255.0

11493376/11490434 [==============================] - 0s 0us/step


##定义模型
我们来构建一个简单的模型，以演示如何保存和加载权重。

In [5]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
      keras.layers.Dense(512, activation=tf.nn.relu, input_shape=(784,)),
      keras.layers.Dropout(0.2),
      keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  
  model.compile(optimizer=tf.keras.optimizers.Adam(),
               loss=tf.keras.losses.sparse_categorical_crossentropy,
               metrics=['accuracy'])
  return model

#Create a basic model instance
model = create_model()
model.summary()

W0822 10:51:34.199566 140017304512384 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


##在训练期间保存检查点
主要用例是，在训练期间或训练结束时自动保存检查点。这样一来，您便可以使用经过训练的模型，而无需重新训练该模型，或从上次暂停的地方继续训练，以防训练过程中断。

`tf.keras.callbacks.ModelCheckpoint `是执行此任务的回调。该回调需要几个参数来配置检查点。

##检查点回调用法
训练模型，并将` ModelCheckpoint `回调传递给该模型：

In [6]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

#Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_weights_only=True,
                                                verbose=1)

model = create_model()

model.fit(train_images, train_labels, epochs = 10,
         validation_data=(test_images, test_labels),
         callbacks = [cp_callback]) # pass callback to training

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 672/1000 [===================>..........] - ETA: 0s - loss: 1.4112 - acc: 0.6027 
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1702 - acc: 0.6740 - val_loss: 0.6920 - val_acc: 0.8050
Epoch 2/10
 704/1000 [====================>.........] - ETA: 0s - loss: 0.4416 - acc: 0.8693
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 179us/sample - loss: 0.4178 - acc: 0.8770 - val_loss: 0.5347 - val_acc: 0.8360
Epoch 3/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2722 - acc: 0.9300
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 176us/sample - loss: 0.2824 - acc: 0.9280 - val_loss: 0.4751 - val_acc: 0.8560
Epoch 4/10
 800/1000 [=======================>......] - ETA: 0s - loss: 0.2121 - acc: 0.9488
Epoch 00004: saving model to training_1/cp.ckpt
1000/100

上述代码将创建一个 TensorFlow 检查点文件集合，这些文件在每个周期结束时更新：

In [7]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00002  cp.ckpt.index
cp.ckpt     cp.ckpt.data-00001-of-00002


创建一个未经训练的全新模型。仅通过权重恢复模型时，您必须有一个与原始模型架构相同的模型。由于模型架构相同，因此我们可以分享权重（尽管是不同的模型实例）。

现在，重新构建一个未经训练的全新模型，并用测试集对其进行评估。未训练模型的表现有很大的偶然性（准确率约为 10%）：

In [8]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

1000/1000 [==============================] - 0s 67us/sample - loss: 2.3671 - acc: 0.0990
Untrained model, accuracy:  9.90%


然后从检查点加载权重，并重新评估：

In [9]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

1000/1000 [==============================] - 0s 42us/sample - loss: 0.3915 - acc: 0.8770
Restored model, accuracy: 87.70%


##检查点回调选项
该回调提供了多个选项，用于为生成的检查点提供独一无二的名称，以及调整检查点创建频率。

训练一个新模型，每隔 5 个周期保存一次检查点并设置唯一名称：

In [10]:
# include the epoch in the file name. (uses 'str.format')
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                verbose=1,
                                                save_weights_only=True,
                                                # Save weights, every 5-epochs.
                                                period=5)

model = create_model()
model.fit(train_images, train_labels,
         epochs=50, callbacks = [cp_callback],
         validation_data=(test_images, test_labels),
         verbose=0)

W0822 11:14:28.159075 140017304512384 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.



Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


现在，看一下生成的检查点并选择最新的检查点：

In [11]:
!ls {checkpoint_dir}

checkpoint			  cp-0030.ckpt.data-00000-of-00002
cp-0005.ckpt.data-00000-of-00002  cp-0030.ckpt.data-00001-of-00002
cp-0005.ckpt.data-00001-of-00002  cp-0030.ckpt.index
cp-0005.ckpt.index		  cp-0035.ckpt.data-00000-of-00002
cp-0010.ckpt.data-00000-of-00002  cp-0035.ckpt.data-00001-of-00002
cp-0010.ckpt.data-00001-of-00002  cp-0035.ckpt.index
cp-0010.ckpt.index		  cp-0040.ckpt.data-00000-of-00002
cp-0015.ckpt.data-00000-of-00002  cp-0040.ckpt.data-00001-of-00002
cp-0015.ckpt.data-00001-of-00002  cp-0040.ckpt.index
cp-0015.ckpt.index		  cp-0045.ckpt.data-00000-of-00002
cp-0020.ckpt.data-00000-of-00002  cp-0045.ckpt.data-00001-of-00002
cp-0020.ckpt.data-00001-of-00002  cp-0045.ckpt.index
cp-0020.ckpt.index		  cp-0050.ckpt.data-00000-of-00002
cp-0025.ckpt.data-00000-of-00002  cp-0050.ckpt.data-00001-of-00002
cp-0025.ckpt.data-00001-of-00002  cp-0050.ckpt.index
cp-0025.ckpt.index


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

>注意：默认的 TensorFlow 格式仅保存最近的 5 个检查点。

要进行测试，请重置模型并加载最新的检查点：

In [13]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 73us/sample - loss: 0.4645 - acc: 0.8780
Restored model, accuracy: 87.80%


##这些是什么文件？
上述代码将权重存储在检查点格式的文件集合中，这些文件仅包含经过训练的权重（采用二进制格式）。检查点包括： * 包含模型权重的一个或多个分片。 * 指示哪些权重存储在哪些分片中的索引文件。

如果您仅在一台机器上训练模型，则您将有 1 个后缀为 .data-00000-of-00001 的分片

##手动保存权重
在上文中，您了解了如何将权重加载到模型中。

手动保存权重的方法同样也很简单，只需使用 Model.save_weights 方法即可。

In [14]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

#Restore the weights
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 73us/sample - loss: 0.4645 - acc: 0.8780
Restored model, accuracy: 87.80%


##保存整个模型
整个模型可以保存到一个文件中，其中包含权重值、模型配置乃至优化器配置。这样，您就可以为模型设置检查点，并稍后从完全相同的状态继续训练，而无需访问原始代码。

在 Keras 中保存完全可正常使用的模型非常有用，您可以在 TensorFlow.js 中加载它们，然后在网络浏览器中训练和运行它们。

Keras 使用 HDF5 标准提供基本的保存格式。对于我们来说，可将保存的模型视为一个二进制 blob。

In [15]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

#Save entire model to a HDF5 file
model.save('my_model.h5')

Epoch 1/5
1000/1000 [==============================] - 0s 150us/sample - loss: 1.1597 - acc: 0.6860
Epoch 2/5
1000/1000 [==============================] - 0s 68us/sample - loss: 0.4242 - acc: 0.8870
Epoch 3/5
1000/1000 [==============================] - 0s 68us/sample - loss: 0.2789 - acc: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 66us/sample - loss: 0.2004 - acc: 0.9500
Epoch 5/5
1000/1000 [==============================] - 0s 68us/sample - loss: 0.1489 - acc: 0.9700


现在，从该文件重新创建模型：

In [17]:
#Recreate the exact same model, including weights and optimizer.
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


检查其准确率：

In [18]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 90us/sample - loss: 0.4188 - acc: 0.8670
Restored model, accuracy: 86.70%


此技巧可保存以下所有内容：

- 权重值
- 模型配置（架构）
- 优化器配置
Keras 通过检查架构来保存模型。目前，它无法保存 TensorFlow 优化器（来自 tf.train）。使用此类优化器时，您需要在加载模型后对其进行重新编译，使优化器的状态变松散。

##后续学习计划
这些就是使用 tf.keras 保存和加载模型的快速指南。

- tf.keras 指南详细介绍了如何使用 tf.keras 保存和加载模型。

- 请参阅在 Eager 中保存，了解如何在 Eager Execution 期间保存模型。

- 保存和恢复指南介绍了有关 TensorFlow 保存的低阶详细信息。